In [2]:
!pip3 install fastapi pandas torch transformers uvicorn google-cloud-storage scikit-learn


  Obtaining dependency information for fastapi from https://files.pythonhosted.org/packages/4d/d2/3ad038a2365fefbac19d9a046cab7ce45f4c7bfa81d877cbece9707de9ce/fastapi-0.103.2-py3-none-any.whl.metadata
  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/1a/d1/3bba59606141ae808017f6fde91453882f931957f125009417b87a281067/transformers-4.34.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.5/121.5 kB 3.9 MB/s eta 0:00:00
  Obtaining dependency information for uvicorn from https://files.pythonhosted.org/packages/79/96/b0882a1c3f7ef3dd86879e041212ae5b62b4bd352320889231cc735a8e8f/uvicorn-0.23.2-py3-none-any.whl.metadata
  Using cached uvicorn-0.23.2-py3-none-any.whl.metadata (6.2 kB)
  Obtaining dependency information for google-cloud-storage from https://files.pythonhosted.org/packages/3a/9f/7923b9e460023470826d124156503359bf77ee130adb0872570599e8cd98/google_cloud_storage-2.11.0-py2.py3-none-any.whl.metadata
  Obtaining d

In [1]:
from fastapi import FastAPI, UploadFile, HTTPException
import pandas as pd
import torch
import json
import io
from transformers import BartTokenizer


/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Load the new JSON files and inspect their structure

with open("../datasets/input_old_model/us-central1_eeg-test_dodadqada_dodadqada_saved_data_input_embeddings_12.json", 'r') as file:
    input_embeddings_data = json.load(file)

with open("../datasets/input_old_model/us-central1_eeg-test_dodadqada_dodadqada_saved_data_input_mask_invert_12.json", 'r') as file:
    input_mask_invert_data = json.load(file)

with open("../datasets/input_old_model/us-central1_eeg-test_dodadqada_dodadqada_saved_data_input_masks_12.json", 'r') as file:
    input_masks_data = json.load(file)

input_embeddings_data, input_mask_invert_data, input_masks_data

([[[-0.5022412209920912,
    -0.4121677088446047,
    -0.5191630601372801,
    -0.8593622599277679,
    -1.020722623419118,
    -1.1012043272468544,
    -0.7363969284356389,
    -0.9308705396152067,
    -0.9638521279496524,
    -0.42165426934170147,
    -0.8897473598925187,
    -1.13664019646623,
    -0.8855217346662599,
    -0.4629453126802669,
    -0.706058745678533,
    -0.4331692574039853,
    -0.7965396793856162,
    -0.29791771564708897,
    -0.3789279110535646,
    -0.37688822116101545,
    -0.2574128926347624,
    -0.4106137274213643,
    -0.6310509861887794,
    -0.5626410276968357,
    -0.9933198701471239,
    -0.4635226580375756,
    -0.42941994621786983,
    -0.48450964797865886,
    -0.7087818665583236,
    -0.7145648244369402,
    -0.7316434022161001,
    -0.4016699561649384,
    -0.5143892067903072,
    -0.5268121032525228,
    -0.3936770549669669,
    -0.40264384532175734,
    -0.48124879217096483,
    -0.45062278827702956,
    -0.3232022448950017,
    -0.49288870965969

In [4]:
import torch.nn as nn
import torch.nn.functional as F
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig

class BrainTranslator(nn.Module):
    def __init__(self, pretrained_layers, in_feature=840, decoder_embedding_size=1024, additional_encoder_nhead=8, additional_encoder_dim_feedforward=2048):
        super(BrainTranslator, self).__init__()
        
        self.pretrained = pretrained_layers
        self.additional_encoder_layer = nn.TransformerEncoderLayer(d_model=in_feature, nhead=additional_encoder_nhead, dim_feedforward=additional_encoder_dim_feedforward, batch_first=True)
        self.additional_encoder = nn.TransformerEncoder(self.additional_encoder_layer, num_layers=6)
        self.fc1 = nn.Linear(in_feature, decoder_embedding_size)

    def forward(self, input_embeddings_batch, input_masks_batch, input_masks_invert):
        encoded_embedding = self.additional_encoder(input_embeddings_batch, src_key_padding_mask=input_masks_invert)
        encoded_embedding = F.relu(self.fc1(encoded_embedding))
        out = self.pretrained(inputs_embeds=encoded_embedding, attention_mask=input_masks_batch, return_dict=True)
        
        return out


In [5]:
from transformers import BartForConditionalGeneration

_MODEL = None

def get_model():
    global _MODEL

    if _MODEL is None:
        pretrained_bart = BartForConditionalGeneration.from_pretrained('facebook/bart-large')
        checkpoint_path = '/Users/michaelholborn/Documents/SoftwareLocal/monotropism/thoughtx/task1_task2_taskNRv2_finetune_BrainTranslator_skipstep1_b1_20_30_5e-05_5e-07_unique_sent.pt'  # Change to the path of your model
        _MODEL = BrainTranslator(pretrained_bart)
        model_weights = torch.load(checkpoint_path, map_location=torch.device('cpu'))
        _MODEL.load_state_dict(model_weights)
        _MODEL.eval()

    return _MODEL


In [6]:
import numpy as np
from sklearn.preprocessing import StandardScaler

def preprocess_eeg_data_for_inference(raw_eeg_data: np.ndarray, segment: bool = False, segment_length: int = 128) -> np.ndarray:
    eeg_data_df = pd.DataFrame(raw_eeg_data)
    eeg_data_filled = eeg_data_df.fillna(eeg_data_df.mean())
    scaler = StandardScaler()
    normalized_data = scaler.fit_transform(eeg_data_filled)
    if segment:
        segmented_data = segment_eeg_data(normalized_data, segment_length)
        return segmented_data
    print("Segment Data: ",segment)
    print("Normalised Data: ",normalized_data)

    return normalized_data

def segment_eeg_data(eeg_data: np.ndarray, segment_length: int = 128) -> np.ndarray:
    num_segments = eeg_data.shape[1] // segment_length
    segmented_data = []
    for i in range(num_segments):
        start_idx = i * segment_length
        end_idx = (i + 1) * segment_length
        segment = eeg_data[:, start_idx:end_idx]
        segmented_data.append(segment)
    return np.array(segmented_data)




In [31]:
# Load data
with open("datasets/results.json", 'r') as file:
    data = json.load(file)

raw_eeg_data = pd.DataFrame(data)

In [20]:

eeg_tensor = preprocess_eeg_data_for_inference(raw_eeg_data)



Segment Data:  False
Normalised Data:  [[-0.12980619 -0.25275585 -0.15944155 ...  0.92220916  0.12201108
  -0.43434176]
 [ 0.02935555 -0.49875623 -0.50513756 ...  1.62593771  0.4048283
  -0.4977017 ]
 [-0.68281424 -1.04184985 -0.89878684 ...  1.64638032 -0.21750464
  -0.85330254]
 ...
 [-0.5933969  -0.83465343 -0.66745245 ...  1.21945275 -0.01502839
  -0.20512466]
 [ 0.20163897 -0.38502061 -0.3968381  ...  1.36628486 -0.14704242
  -0.5807963 ]
 [-0.55645603 -0.13538309  0.14307366 ...  0.37879432  0.09646755
  -0.3338418 ]]


In [7]:
model =get_model()
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')



FileNotFoundError: [Errno 2] No such file or directory: '/Users/michaelholborn/Documents/SoftwareLocal/monotropism/thoughtx/task1_task2_taskNRv2_finetune_BrainTranslator_skipstep1_b1_20_30_5e-05_5e-07_unique_sent.pt'

In [26]:

# def generate_text_from_eeg(input_sample: dict, model, tokenizer, device="cpu") -> str:
#     """
#     Generate text from preprocessed EEG data using a trained model.
    
#     Parameters:
#     - input_sample: The prepared input sample.
#     - model: The trained EEG-to-text model.
#     - tokenizer: The BART tokenizer.
#     - device: The device to run the model on (e.g., "cpu", "cuda").
    
#     Returns:
#     - Generated text.
#     """
    
#     # Move the sample and model to the specified device
#     input_sample["sent_level_EEG"] = input_sample["sent_level_EEG"].to(device)
#     input_sample["target_ids"] = input_sample["target_ids"].to(device)
#     model = model.to(device)
    
#     # Set the model to evaluation mode
#     model.eval()
    
#     print("input_sample: ",input_sample)
    
#     # Perform inference
#     with torch.no_grad():
#         outputs = model(input_ids=None, encoder_outputs=(input_sample["sent_level_EEG"], None), decoder_input_ids=input_sample["target_ids"])
    
#     # Extract the generated token IDs from the model's outputs
#     generated_ids = outputs.logits.argmax(dim=-1)
    
#     # Decode the token IDs to text
#     generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    
#     return generated_text

In [28]:
def generate_text_from_eeg(input_sample: dict, model, tokenizer, device="cpu") -> str:
    """
    Generate text from preprocessed EEG data using a trained model.
    
    Parameters:
    - input_sample: The prepared input sample.
    - model: The trained EEG-to-text model.
    - tokenizer: The BART tokenizer.
    - device: The device to run the model on (e.g., "cpu", "cuda").
    
    Returns:
    - Generated text.
    """
    
    # Move the sample and model to the specified device
    input_sample["sent_level_EEG"] = input_sample["sent_level_EEG"].to(device)
    input_sample["target_ids"] = input_sample["target_ids"].to(device)
    model = model.to(device)
    
    # Set the model to evaluation mode
    model.eval()
    
    # Perform inference
    with torch.no_grad():
        outputs = model(input_sample["sent_level_EEG"], input_sample["target_ids"], None)  # Adjusted this line to match the model's forward method
    
    # Extract the generated token IDs from the model's outputs
    generated_ids = outputs.logits.argmax(dim=-1)
    
    # Decode the token IDs to text
    generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    
    return generated_text


### Step 1: Ensure input_sample is structured correctly
Before calling generate_text_from_eeg, the input_sample should be structured as a dictionary with keys sent_level_EEG and target_ids. We need to check if this is the case.

In [8]:
import torch

# Convert loaded data to PyTorch tensors
input_embeddings_tensor = torch.tensor(input_embeddings_data)
input_masks_tensor = torch.tensor(input_masks_data)
input_mask_invert_tensor = torch.tensor(input_mask_invert_data)

input_embeddings_tensor.shape, input_masks_tensor.shape, input_mask_invert_tensor.shape

(torch.Size([1, 56, 840]), torch.Size([1, 56]), torch.Size([1, 56]))

In [47]:
input_embeddings_tensor

tensor([[[-0.3485, -0.6535, -0.5522,  ...,  1.0953,  1.0049, -1.5333],
         [-0.4472, -0.3577, -0.2128,  ...,  1.1794,  1.0307, -1.3763],
         [ 1.0523,  0.4421,  0.1681,  ...,  0.2377,  0.3593, -1.8879],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]])

In [48]:
input_masks_tensor

tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.]])

In [9]:
# Model-related imports and definitions
from transformers import BartTokenizer, BartForConditionalGeneration
import torch.nn as nn

# Define the BrainTranslator model as provided in the user's code
class BrainTranslator(nn.Module):
    def __init__(self, pretrained_layers, in_feature=840, decoder_embedding_size=1024, additional_encoder_nhead=8, additional_encoder_dim_feedforward=2048):
        super(BrainTranslator, self).__init__()
        
        self.pretrained = pretrained_layers
        self.additional_encoder_layer = nn.TransformerEncoderLayer(d_model=in_feature, nhead=additional_encoder_nhead, dim_feedforward=additional_encoder_dim_feedforward, batch_first=True)
        self.additional_encoder = nn.TransformerEncoder(self.additional_encoder_layer, num_layers=6)
        self.fc1 = nn.Linear(in_feature, decoder_embedding_size)

    def forward(self, input_embeddings_batch, input_masks_batch, input_masks_invert):
        encoded_embedding = self.additional_encoder(input_embeddings_batch, src_key_padding_mask=input_masks_invert)
        encoded_embedding = F.relu(self.fc1(encoded_embedding))
        out = self.pretrained(inputs_embeds=encoded_embedding, attention_mask=input_masks_batch, return_dict=True)
        
        return out

# Initialize model
pretrained_bart = BartForConditionalGeneration.from_pretrained('facebook/bart-large')
model = BrainTranslator(pretrained_bart)

# Attempt a forward pass
with torch.no_grad():
    try:
        outputs = model(input_embeddings_tensor, input_masks_tensor, input_mask_invert_tensor)
        result = "Forward pass successful!"
    except Exception as e:
        result = str(e)

result


'If no `decoder_input_ids` or `decoder_inputs_embeds` are passed, `input_ids` cannot be `None`. Please pass either `input_ids` or `decoder_input_ids` or `decoder_inputs_embeds`.'

In [60]:
from transformers import BartTokenizer, BartForConditionalGeneration
import torch.nn as nn
import torch.nn.functional as F

# Define the BrainTranslator model
class BrainTranslator(nn.Module):
    def __init__(self, pretrained_layers, in_feature=840, decoder_embedding_size=1024, additional_encoder_nhead=8, additional_encoder_dim_feedforward=2048):
        super(BrainTranslator, self).__init__()
        
        self.pretrained = pretrained_layers
        self.additional_encoder_layer = nn.TransformerEncoderLayer(d_model=in_feature, nhead=additional_encoder_nhead, dim_feedforward=additional_encoder_dim_feedforward, batch_first=True)
        self.additional_encoder = nn.TransformerEncoder(self.additional_encoder_layer, num_layers=6)
        self.fc1 = nn.Linear(in_feature, decoder_embedding_size)

    def forward(self, input_embeddings_batch, input_masks_batch, input_mask_invert, decoder_input_ids):
        encoded_embedding = self.additional_encoder(input_embeddings_batch, src_key_padding_mask=input_mask_invert)
        encoded_embedding = F.relu(self.fc1(encoded_embedding))
        out = self.pretrained(inputs_embeds=encoded_embedding, attention_mask=input_masks_batch, decoder_input_ids=decoder_input_ids, return_dict=True)
        return out

# Initialize model
pretrained_bart = BartForConditionalGeneration.from_pretrained('facebook/bart-large')
model = BrainTranslator(pretrained_bart)

# Create a placeholder token
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
placeholder_token = tokenizer("<s>", return_tensors="pt")

# Attempt a forward pass
with torch.no_grad():
    try:
        outputs = model(input_embeddings_tensor, input_masks_tensor, input_mask_invert_tensor, placeholder_token["input_ids"])
        result = "Forward pass successful!"
    except Exception as e:
        result = str(e)

result
outputs
# Extract the generated token IDs from the model's outputs
generated_ids = outputs.logits.argmax(dim=-1)
generated_ids

# Decode the token IDs to text
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
tokenizer.decode(generated_ids[0], skip_special_tokens=True)



'Forward pass successful!'

In [61]:
outputs

Seq2SeqLMOutput(loss=None, logits=tensor([[[16.5298, -1.6600,  5.7722,  ..., -4.1267, -4.3760,  4.4471],
         [16.5298, -1.6600,  5.7722,  ..., -4.1267, -4.3760,  4.4471],
         [16.5264, -1.6601,  5.7720,  ..., -4.1264, -4.3757,  4.4469]]]), past_key_values=((tensor([[[[ 1.1687,  2.8813,  4.4058,  ..., -1.6669,  3.0377, -2.8006],
          [-0.0745,  0.6451,  1.7084,  ...,  0.9115,  2.5264, -1.6802],
          [-2.5676, -2.6669, -4.6881,  ..., -1.6383, -2.6659, -2.1031]],

         [[-1.3328, -4.4495,  1.5480,  ..., -1.8666,  0.8618, -0.2465],
          [-1.8701, -2.2585, -0.1146,  ..., -1.3593, -0.1351,  0.1502],
          [-3.3394, -3.4319,  1.0716,  ..., -4.8966, -1.3012,  4.3711]],

         [[ 0.1951,  0.9602,  0.9443,  ..., -0.1951, -1.0370,  2.6584],
          [-0.0807,  0.2310,  0.5215,  ..., -0.2660, -0.5006,  3.1165],
          [ 0.7811,  1.6962,  2.5301,  ..., -1.8475, -0.5803, -0.0753]],

         ...,

         [[ 0.9762,  1.0852,  0.8219,  ...,  1.1969,  1.7232,  

In [62]:
# Extract the generated token IDs from the model's outputs
generated_ids = outputs.logits.argmax(dim=-1)
generated_ids


tensor([[0, 0, 0]])

In [63]:

# Decode the token IDs to text
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
tokenizer.decode(generated_ids[0], skip_special_tokens=True)

''

In [53]:
for x in range(0,10):
    print(x)
    generated_text = tokenizer.decode(generated_ids[x], skip_special_tokens=True)
    print(generated_text)
    



0

1


IndexError: index 1 is out of bounds for dimension 0 with size 1

In [45]:
generated_text

''

In [29]:

# Preprocess the data
placeholder_token = tokenizer("<s>", return_tensors="pt")
input_sample = {
    "sent_level_EEG": torch.tensor(eeg_tensor),
    "target_ids": placeholder_token["input_ids"]
}

results_json = generate_text_from_eeg(input_sample,model,tokenizer)


AssertionError: was expecting embedding dimension of 840, but got 1140